<a href="https://www.kaggle.com/code/eleonoraricci/ps3e14-berrylicious-predictions-with-pytorch?scriptVersionId=128619927" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **PyTorch🔥 model and FLAML🔥 hyperparameter tuning**

In this Notebook I wanted to share a **baseline Neural Network** implementation in **PyTorch** to model the *Wild Blueberry Yield Dataset* for the episode 14 of season 3 of the playGround competitions series on Kaggle.

The architecture consists of a sequence of densely connected layers. **EarlyStopping** and **Dropout** are implemented and can be tinkered with by changing the corresponding parameters in the code.
_____________________________

*--- Edit 23/05/06: implemented **automatic HP search with FLAML**. Main inspirations: **[this link](https://microsoft.github.io/FLAML/docs/Examples/Tune-PyTorch/)** and **[this notebook](https://www.kaggle.com/code/paddykb/ps-s3e14-flaml-bfi-be-bop-a-blueberry-do-dah/notebook)**. ---*
_____________________________

**Preliminary observation:** just by scaling the features ([StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.htmlhttps://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)) **the MAE went down by ~50**.
_____________________________
Now off to some feature engineering, and some more hyperparameter tuning 👋

Any feedback or question is more than welcome!! 🧐

In [1]:
%%capture
!pip install flaml

In [2]:
import time
import os

import numpy as np
import pandas as pd

from typing import List, Callable

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
torch.set_default_dtype(torch.float32)

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

from ray import tune
import flaml

seed = 17
torch.manual_seed(seed)
np.random.seed(seed)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# **Neural network and auxiliary classes**

In [3]:
class NN(nn.Module):
    """Simple densely connected neural network model"""
    
    def __init__(self, n_hidden_layers: int, 
                       input_dim: int, 
                       hidden_dims: List[int], 
                       output_dim:int, 
                       dropout_prob: float,
                       activation: str):
        
        """      
        Parameters
        ----------
        n_hidden_layers: int
            The number of hidden layers to include between input and output

        input_dim: int
            Number of nodes in the input layer. It is equal to the number of 
            features used for modelling.
            
        hidden_dims: List[int]
            List of ints specifying the number of nodes in each hidden layer 
            of the network.
            
        output_dim: int
            Number of nodes in the output layer. It is equal to 1 for 
            this regression task.
            
        dropout_prob: float
            value between 0 and 1: probability of zeroing out elements of the 
            input tensor in the Dropout layer
            
        activation: str
            the name of the activation function from the torch.nn module is 
            passed as a string and then resolved by getattr(nn, activation)()

        """

        super(NN, self).__init__()
        
        # The Network is built using the ModuleList constructor        
        self.network = nn.ModuleList()
        
        if n_hidden_layers == 0:
            # If no hidden layers are used, go directly from input to 
            # output dimension
            self.network.append(nn.Linear(input_dim, output_dim))
            self.network.append(getattr(nn, activation)())
        
        else:
            # Append to the constructor the required layers, with dimentions specified
            # in the hidden_dims list
            if dropout_prob > 0:
                self.network.append(torch.nn.Dropout(p=dropout_prob, inplace=True))
            self.network.append(nn.Linear(input_dim, hidden_dims[0]))
            self.network.append(nn.LayerNorm(hidden_dims[0]))
            self.network.append(getattr(nn, activation)())

            for layer in range(n_hidden_layers-1):
                self.network.append(nn.Linear(hidden_dims[layer], hidden_dims[layer+1]))
                self.network.append(nn.LayerNorm(hidden_dims[layer+1]))
                self.network.append(getattr(nn, activation)())

            self.network.append(nn.Linear(hidden_dims[-1], output_dim))
            self.network.append(nn.ReLU())
    
    def forward(self, x):
        for layer in self.network:
            x = layer(x)  
            
        return x
    

def get_model(n_features: int, 
              hidden_dims: List[int], 
              activation: str, 
              dropout_prob: float):
    
    """Return a newly initialized NN model to perform HP tuning rounds
    
    Parameters
    ----------
            
    hidden_dims: List[int]
        List of ints specifying the number of nodes in each hidden layer 
        of the network. e.g. hidden_dims = [100, 100, 100]
        
    activation: str
        the name of the activation function from the torch.nn module is 
        passed as a string and then resolved by getattr(nn, activation)()
        
    dropout_prob: float
        value between 0 and 1: probability of zeroing out elements of the 
        input tensor in the Dropout layer
    """ 
    
    model = NN(n_hidden_layers = len(hidden_dims), 
               input_dim = n_features, 
               hidden_dims = hidden_dims, 
               output_dim = 1, 
               activation = activation, 
               dropout_prob = dropout_prob)
    
    return model

In [4]:
class EarlyStopping():
    """
    Early stopping to stop the training when the validation loss does not improve after
    a certain number of epochs.
    """
    def __init__(self, patience=5, min_delta=0.1):
        """
        Parameters
        ----------
        
        patience: how many epochs to wait before stopping when loss is not improving
               
        min_delta: minimum difference between new loss and old loss for new loss to 
               be considered as an improvement
        """
        
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
            
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            # reset counter if validation loss improves
            self.counter = 0
            
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [5]:
class MyDataset(Dataset):
    """A Custom Dataset class to facilitate batch iteration in PyTorch models"""
    def __init__(self, 
                 features, 
                 labels):

        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):

        label = self.labels[idx]
        features = self.features[idx,:]

        return features, label

In [6]:
class ColumnsDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        super(ColumnsDropper, self).__init__()
        self.columns=columns

    def transform(self,X, y=None):
        return X.drop(self.columns,axis=1)

    def fit(self, X, y=None):
        return self

# **Preprocessing pipeline**

In [7]:
training_set = pd.read_csv(r'../input/playground-series-s3e14/train.csv')
orig_train = pd.read_csv(r'../input/wild-blueberry-yield-prediction-dataset/WildBlueberryPollinationSimulationData.csv')
orig_train.drop('Row#', axis = 1, inplace = True)

training_set.drop('id', axis = 1, inplace = True)
combined_train = pd.concat([training_set, orig_train])
X = combined_train.copy()

y = X.pop('yield')

In [8]:
# Minimal preprocessing, appling standard scaling to all features
columns_to_drop = ['MaxOfUpperTRange', 'MinOfUpperTRange', 'MaxOfLowerTRange', 'MinOfLowerTRange', 'RainingDays']
all_features = list(X.columns)
standard_scale = [feat for feat in all_features if feat not in columns_to_drop]

In [9]:
col_transformers=[
    ('std', StandardScaler(), standard_scale),
    # More transformers can be added as needed. 
    # In ColumnTransformer one can specify different columns 
    # lists for each preprocessing action to perform. 
]

preprocessor = ColumnTransformer(
    transformers=col_transformers,      
    remainder='passthrough')

preproc_pipeline = Pipeline(steps=[('dropper', ColumnsDropper(columns = columns_to_drop)),
                                   ('preprocessor', preprocessor)])

X_proc = preproc_pipeline.fit_transform(X)  

In [10]:
test_size = 0.2 

# Splitting into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X_proc, y.to_numpy(), test_size = test_size, random_state = seed)

# **Hyperparameter search space**

In [11]:
# Search space for HP tuning. 
# This is a small set, to allow for the completion of a few tests within a short 
# execution time span, to troubleshoot. It can be adjusted as needed for production.

config = {
    "layer1": tune.choice([200]),
    "layer2": tune.choice([200]),
    "layer3": tune.choice([200]),  
    "lr": tune.choice([0.01]),
    "batch_size": tune.choice([100]),
    "activation": tune.choice(["ReLU", "Tanh"]),
    "dropout_prob": tune.choice([0, 0.2]), 
}

# Budget and resource constraints
time_budget_s = 600     # time budget in seconds
num_samples = 100       # maximal number of trials

# Early stopping parameters
patience = 20
min_change = 0.0001


# **Training loop**

In [12]:
# The training loop is wrapped inside a function, to be able to pass 
# this fuction as an argument for HP tuning.

def train_nn(config):
    
    # Input size of the network equal to the number of retained feature (here all features)
    n_features = X_train.shape[1]
    
    # Creating Datasets and DataLoaders for batch iteration
    training_dataset = MyDataset(X_train, y_train)
    val_dataset = MyDataset(X_val, y_val)

    train_loader = DataLoader(training_dataset, batch_size=config["batch_size"], shuffle=True) 
    val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=True)
    
    hidden_dims = [config["layer1"], config["layer2"], config["layer3"]]
    model = get_model(n_features, hidden_dims, config["activation"], config["dropout_prob"])
    
    # MAE loss in PyTorch
    loss_function = torch.nn.L1Loss()
    
    # Initialization of the losses, instantiation of optimizer, early_stopper and learning rate scheduler
    loss_epoch = torch.tensor(0.0)
    loss_validation_epoch = torch.tensor(0.0)

    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    early_stopper = EarlyStopping(patience = patience, min_delta = min_change)

    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
                    optimizer = optimizer, mode='min', patience = 10,
                    factor = 0.1, min_lr=1e-5, verbose=True)
                    
    # A high number of epochs can be used, since ealry stopping can interrupt the training if needed
    n_epochs = 500

    for epoch in range(n_epochs):

            # (re)initialize the tensor to accumulate batch loss values
            loss_batch = torch.tensor(0.0)

            for i, data in enumerate(train_loader):
                # Every 'data' instance returned by the train_loader is an input + label pair
                inputs, labels = data

                # Zero gradients for every batch
                optimizer.zero_grad()

                # Make predictions for the current batch
                outputs = model(inputs.type(torch.float32))

                # Compute the loss and perform back propagation
                loss = loss_function(outputs.squeeze(), labels)
                loss.backward()
                optimizer.step()

                # Accumulate the loss over the batches
                loss_batch += loss.item()

            # Average the loss over the number of batches
            loss_epoch = loss_batch / i

            # (re)initialize the tensor to accumulate loss values for the validation set
            loss_validation_batch = torch.tensor(0.0)

            for i, data in enumerate(val_loader):
                # Make prediction for each batch in the test set
                inputs, labels = data
                outputs_val = model(inputs.type(torch.float32))

                # Accumulate the loss over the test set batches
                loss_val = loss_function(outputs_val.squeeze(), labels)
                loss_validation_batch += loss_val.item()

            # Average the loss over the number of batches
            loss_validation_epoch = loss_validation_batch / i

            print("Epoch: ", epoch +1 , " - Train Loss: ", round(loss_epoch.item(), 4), " - Val Loss: ", round(loss_validation_epoch.item(),4))

            # Here we save a checkpoint. It is automatically registered with
            # Ray Tune and will potentially be passed as the `checkpoint_dir`
            # parameter in future iterations.
            with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                torch.save(
                    (model.state_dict(), optimizer.state_dict()), path)

            tune.report(loss=loss_validation_epoch.item())
            # Update learning rate scheduler and check early stoopping criterun
            lr_scheduler.step(loss_validation_epoch)

            early_stopper(loss_validation_epoch)
            if early_stopper.early_stop:
                break
            

# **HP optimization rounds**

In [13]:
# Time the round
start_time = time.time()

result = flaml.tune.run(tune.with_parameters(train_nn),   # The training loop is wrapped inside function train_nn
                        config = config,                  # The HP search space dictionary
                        metric = "loss",                  # Metric to chose the best case. It is stored in tune.report within train_nn
                        mode = "min",                     # Lower is better
                        resources_per_trial = {"cpu": 4}, # Define usable hardware
                        local_dir = 'logs/',              # Set output directory
                        num_samples = num_samples,        # Search budget: maximum number of tests
                        time_budget_s = time_budget_s,    # Search budget: maximum time
                        use_ray=True)

[I 2023-05-07 09:01:04,988] A new study created in memory with name: optuna
2023-05-07 09:01:09,493	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Trial name,loss,should_checkpoint
train_nn_47b1c36d,385.613,True
train_nn_55cbb65b,368.056,True
train_nn_8adc96d7,388.712,True
train_nn_9f24a79e,361.027,True


(train_nn pid=424) Epoch:  1  - Train Loss:  5932.3955  - Val Loss:  5943.0347
(train_nn pid=424) Epoch:  2  - Train Loss:  5652.9028  - Val Loss:  5663.5664
(train_nn pid=424) Epoch:  3  - Train Loss:  5384.3325  - Val Loss:  5393.2368
(train_nn pid=424) Epoch:  4  - Train Loss:  5119.8896  - Val Loss:  5119.3677
(train_nn pid=424) Epoch:  5  - Train Loss:  4855.2822  - Val Loss:  4873.5391
(train_nn pid=424) Epoch:  6  - Train Loss:  4593.0088  - Val Loss:  4579.7681
(train_nn pid=424) Epoch:  7  - Train Loss:  4329.7129  - Val Loss:  4312.9668
(train_nn pid=424) Epoch:  8  - Train Loss:  4065.3911  - Val Loss:  4043.5637
(train_nn pid=424) Epoch:  9  - Train Loss:  3804.9927  - Val Loss:  3772.7954
(train_nn pid=424) Epoch:  10  - Train Loss:  3545.9878  - Val Loss:  3523.8188
(train_nn pid=424) Epoch:  11  - Train Loss:  3289.1509  - Val Loss:  3266.7354
(train_nn pid=424) Epoch:  12  - Train Loss:  3037.2075  - Val Loss:  3015.0977
(train_nn pid=424) Epoch:  13  - Train Loss:  278

2023-05-07 09:15:34,660	INFO timeout.py:54 -- Reached timeout of 600 seconds. Stopping all trials.
2023-05-07 09:15:34,768	INFO tune.py:798 -- Total run time: 862.81 seconds (862.49 seconds for the tuning loop).


In [14]:
# Print out outcome of the round: number of trials and time
print(f"Number of trials = {len(result.trials)}")
print(f"Time = {time.time()-start_time} seconds")

# Extract the best case
best_trial = result.get_best_trial("loss", "min", "all")

# Give some info on the best result
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
    best_trial.metric_analysis["loss"]["min"]))

Number of trials = 4
Time = 869.8790180683136 seconds
Best trial config: {'layer1': 200, 'layer2': 200, 'layer3': 200, 'lr': 0.01, 'batch_size': 100, 'activation': 'ReLU', 'dropout_prob': 0}
Best trial final validation loss: 357.69793701171875


# **Predictions on the test set and submission**

In [15]:
# Extract best hyperparameters and initialize a new model with these values
n_features = X_train.shape[1]

best_hidden_dims = [best_trial.config["layer1"], best_trial.config["layer2"], best_trial.config["layer3"]]
best_trained_model = get_model(n_features, best_hidden_dims, best_trial.config["activation"], best_trial.config["dropout_prob"])

# From the saved checkpoint load the state of the best model for further predictions
checkpoint_value = getattr(best_trial.checkpoint, "dir_or_data", None) or best_trial.checkpoint.value
checkpoint_path = os.path.join(checkpoint_value, "checkpoint")

model_state, optimizer_state = torch.load(checkpoint_path)
best_trained_model.load_state_dict(model_state)

<All keys matched successfully>

In [16]:
submission = pd.read_csv(r'../input/playground-series-s3e14/test.csv')
ids = submission.pop('id')

# Apply scaling to the features of the test set
submission_proc = preproc_pipeline.transform(submission)  

predictions = best_trained_model(torch.tensor(submission_proc).type(torch.float32))

result = pd.DataFrame(torch.hstack((torch.tensor(ids.to_numpy().reshape((-1, 1))), predictions.detach())), columns=['id','yield'])
result['id'] = result['id'].astype(int)
result.to_csv('submission.csv',index =False)